In [ ]:
import pandas as pd
import numpy as np
from tqdm import tqdm
from sklearn.model_selection import KFold
from sklearn.metrics import classification_report, f1_score,accuracy_score
from collections import Counter
import warnings
from pandasgui import show
import gc
import catboost as cb
import ipywidgets
from sklearn.ensemble import BaggingClassifier
from collections import Counter
from scipy import stats
warnings.filterwarnings("ignore")

In [ ]:
attr = pd.read_csv('attr.txt', sep='\t',
                       names=['link', 'length', 'direction', 'path_class', 'speed_class', 'LaneNum', 'speed_limit',
                              'level', 'width'], header=None)

In [ ]:
pro = pd.read_csv('per_road_pro.txt')

In [ ]:
ver = pd.read_csv('train/is_train_20190730.txt')
#ver['is_night'] = ver['future_slice_id'].apply(lambda x: 0 if (0< x <= 172) | (x>=690) else 1)
ver = ver.merge(attr, on='link', how='left')
ver = ver.merge(pro, on='link', how='left')

In [ ]:
use_cols = [i for i in ver.columns if i not in ['link', 'label', 'label_pred']]
X_ver = (ver.loc[:,use_cols])
y_ver = (ver['label'])

In [ ]:
class_ = []

In [ ]:
test = pd.read_csv('testgai.txt')

test = test.merge(pro, on='link', how='left')
X_test= (test.loc[:,use_cols])
y_test = (test['label'])

## 多个单天bagging

In [ ]:
from sklearn.ensemble import BaggingClassifier
for i in tqdm(range(20,30)):
    path = 'train/is_train_201907'+str(i)+'.txt'
    data = pd.read_csv(path)
    #data['is_night'] = data['future_slice_id'].apply(lambda x: 0 if (0< x <= 172) | (x>=690) else 1)
    data = data.merge(attr, on='link', how='left')
    data = data.merge(pro, on='link', how='left')
    X_train= (data.loc[:,use_cols])
    y_train = (data['label'])
    tree = cb.CatBoostClassifier(iterations=100,depth=5,l2_leaf_reg=8,learning_rate=0.5,random_strength=1,losns_function='MultiClassOneVsAll',logging_level='Verbose')
    bag = BaggingClassifier(base_estimator=tree,n_estimators=6,max_samples=1.0,max_features=1.0,bootstrap=True,
                            bootstrap_features=False,n_jobs=1,random_state=1)
    w = np.array((y_train+1))
    bag = bag.fit(X_train,y_train,sample_weight=w)
    class_.append(bag)
    y_train_pred = bag.predict(X_train)
    y_test_pred = bag.predict(X_ver)
    test_pred = bag.predict(X_test)
    if (i==20):
        sum_ = y_test_pred
        sum_test = test_pred
        i = 0
    else:
        sum_ = np.vstack((sum_,y_test_pred))
        sum_test = np.vstack((sum_test,test_pred))
    print('sum_:',sum_.shape)
    bag_train = accuracy_score(y_train,y_train_pred)
    bag_test = accuracy_score(y_ver,y_test_pred)
    print('Bagging train/test accuracies %.3f/%.3f' % (bag_train,bag_test))

In [ ]:
from joblib import dump
dump(class_, "class_.txt")  

In [ ]:
temp = stats.mode(sum_,axis=0)

In [ ]:
fin_res = temp.mode.reshape(-1)

In [ ]:
sub_res = stats.mode(sum_test,axis=0)
fin_res = sub_res.mode.reshape(-1)

In [ ]:
print(len(np.where(fin_res==0)[0]),len(np.where(fin_res==1)[0]),len(np.where(fin_res==2)[0]))

In [ ]:
i = 0
for cla in tqdm(class_):
    temp_res = cla.predict_proba(X_test)
    if (i==0):
        fin = temp_res
        i=2
    else:
        #fin = np.vstack((fin,temp_res))
        fin = fin+temp_res

In [ ]:
ah = fin.copy()

In [ ]:
fin = ah.copy()
fin = fin/fin.sum(axis=1).reshape(-1,1)

In [ ]:
len(np.where(fin[:,0]>0.62)[0])

In [ ]:
len(np.where(fin[:,1]>0.34)[0])

In [ ]:
len(np.where(fin[:,2]>0.3)[0])

In [ ]:
a = (np.zeros((176057,1))-1)

In [ ]:
#len(np.where(fin[:,2]>0.33)[0])
a[np.where(fin[:,2]>0.3)[0]]=3
fin[np.where(fin[:,2]<=0.3)[0],2]=0

In [ ]:
a[np.where(fin[:,0]>0.62)[0]]=1
fin[np.where(fin[:,0]<=0.62)[0],0]=0

In [ ]:
fin

In [ ]:
a = np.where(a==-1,2,a)

In [ ]:
len(np.where(a==1)[0])

In [ ]:
len(np.where(a==2)[0])

In [ ]:
len(np.where(a==3)[0])

In [ ]:
lgb = test[['link', 'current_slice_id', 'future_slice_id', 'label']]

In [ ]:
lgb['label'] = a

In [ ]:
lgb.to_csv('2020_11_22_1.csv', index=False, encoding='utf8')

In [ ]:
test1 = pd.read_csv('time_slice/one_day.txt')
test2 = pd.read_csv('time_slice/two_day.txt')
test = pd.concat((test1,test2))

In [ ]:
#test['is_night'] = test['future_slice_id'].apply(lambda x: 0 if (0< x <= 172) | (x>=690) else 1)
test = test.merge(attr, on='link', how='left')
test = test.merge(pro, on='link', how='left')

In [ ]:
X_test = (test.loc[:,use_cols])
y_test = (test['label'])

In [ ]:
i = 0
for cla in tqdm(class_):
    temp_res = cla.predict(X_test)
    if (i==0):
        fin = temp_res
        i=2
    else:
        fin = np.vstack((fin,temp_res))

In [ ]:
temp = stats.mode(fin,axis=0)

In [ ]:
fin_res = temp.mode.reshape(-1)

In [ ]:
print(classification_report(y_test,fin_res,digits=4))
report = f1_score(y_test,fin_res, average=None)
print('Score: ', report[0] * 0.2 + report[1] * 0.2 + report[2] * 0.6)

In [ ]:
lgb = test[['link', 'current_slice_id', 'future_slice_id', 'label']]

In [ ]:
lgb['label']=fin_res+1

In [ ]:
lgb.to_csv('2020_11_17.csv', index=False, encoding='utf8')

## 34万bagging